# Assessing Campaign Performance using Chi-Square Test for Independence

## Problem Statement/Project Overview

Earlier in the year, our client, a grocery retailer, ran a campaign to promote their new "Delivery Club" - an initiative that costs a customer 100 dollars per year for membership, but offers free grocery deliveries rather than the normal cost of 10 dollars per delivery.

For the campaign promoting the club, customers were put randomly into three groups - the first group received a low quality, low cost mailer, the second group received a high quality, high cost mailer, and the third group were a control group, receiving no mailer at all.

The client knows that customers who were contacted, signed up for the Delivery Club at a far higher rate than the control group, but now want to understand if there is a significant difference in signup rate between the cheap mailer and the expensive mailer.  This will allow them to make more informed decisions in the future, with the overall aim of optimising campaign ROI!



## Approach

To do this, we apply Chi-Square Test For Independence (a Hypothesis Test) to assess the performance of two types of mailers that were sent out to promote a new service! 

For this test, as it is focused on comparing the *rates* of two groups - we applied the Chi-Square Test For Independence.  Full details of this test can be found in the dedicated section below.


### Import and understand data

In [2]:
import pandas as pd
from scipy.stats import chi2_contingency, chi2

In [4]:
campaign_data = pd.read_excel("C:/Users/Ibiene/OneDrive/DataScience_MachineLearning/Data Science Infinity/Machine Learning/Model Building/data/grocery_database.xlsx", sheet_name = 'campaign_data')

In [5]:
campaign_data.head()

customer_id  campaign_name campaign_date mailer_type  signup_flag
0           74  delivery_club    2020-07-01     Mailer1            1
1          524  delivery_club    2020-07-01     Mailer1            1
2          607  delivery_club    2020-07-01     Mailer2            1
3          343  delivery_club    2020-07-01     Mailer1            0
4          322  delivery_club    2020-07-01     Mailer2            1

In [6]:
#remove customers who were in the control group
campaign_data.mailer_type.value_counts()

Mailer1    375
Mailer2    336
Control    159
Name: mailer_type, dtype: int64

In [7]:
campaign_data = campaign_data[campaign_data.mailer_type != "Control"]

In [8]:
campaign_data.mailer_type.value_counts()

Mailer1    375
Mailer2    336
Name: mailer_type, dtype: int64

In [31]:
pd.pivot_table(campaign_data, index = ['mailer_type'], values = ['signup_flag'])



signup_flag
mailer_type             
Mailer1         0.328000
Mailer2         0.377976

Mailer 2 appears to have a higher sign up flag of 37.8% - is that significantly different from Mailer 1 at 32.8% or is it random chance?

In [29]:
pd.pivot_table(campaign_data, index = ['mailer_type'], values = ['customer_id'], aggfunc = 'count')

customer_id
mailer_type             
Mailer1              375
Mailer2              336

### Establish null, alternate hypotheses, and acceptance criteria

In [10]:
null_hypothesis = "There is no relationship between mailer type and signup rate. They are independent"
alternate_hypothesis = "There is a relationship between mailer type and signup rate. They are not independent"
acceptance_criteria = 0.05

### Calculate observed frequencies and expected frequencies. 
Note that **observed frequencies** are the true values we've seen, that is, the actual rates per group in the data itself. 
The **expected frequencies** are what we would expect to see based on all of the data combined.

The code below summarises the dataset to a 2x2 matrix for **signup_flag* by **mailer_type**


In [41]:
observed_values = pd.crosstab(campaign_data.mailer_type, campaign_data.signup_flag).values
observed_values

array([[252, 123],
       [209, 127]], dtype=int64)

In [42]:
chi2_statistic, p_value, dof, expected_values = chi2_contingency(observed_values, correction = False)

**Note** When applying the Chi-Square Test above, we use the parameter *correction = False* which means we are applying what is known as the *Yate's Correction* which is applied when your Degrees of Freedom is equal to one.  This correction helps to prevent overestimation of statistical signficance in this case.


### Print Results and Interpretation

In [38]:
print(chi2_statistic)

1.9414468614812481


In [15]:
print(p_value)

0.16351152223398197


In [16]:
critical_value = chi2.ppf(1-acceptance_criteria, dof)

In [17]:
print(critical_value)

3.841458820694124


In [18]:
# print results based on p-value
if p_value <= acceptance_criteria:
    print(f"As our p-value of {p_value} is lower than our acceptance_criteria of {acceptance_criteria} - we reject the null hypothesis, and conclude that: {alternate_hypothesis}")
else:
    print(f"As our p-value of {p_value} is higher than our acceptance_criteria of {acceptance_criteria} - we retain the null hypothesis, and conclude that: {null_hypothesis}")


As our p-value of 0.16351152223398197 is higher than our acceptance_criteria of 0.05 - we retain the null hypothesis, and conclude that: There is no relationship between mailer type and signup rate. They are independent


In [20]:
# print results based on chi2-value
if chi2_statistic >= critical_value:
    print(f"As our chi-square statistic of {chi2_statistic} is higher than our critical value of {critical_value} - we reject the null hypothesis, and conclude that: {alternate_hypothesis}")
else:
    print(f"As our chi-square statistic of {chi2_statistic} is lower than our critical value of {critical_value} - we retain the null hypothesis, and conclude that: {null_hypothesis}")
    

As our chi-square statistic of 1.9414468614812481 is lower than our critical value of 3.841458820694124 - we retain the null hypothesis, and conclude that: There is no relationship between mailer type and signup rate. They are independent


### Discussion and Conclusion

As we can see from the outputs of these print statements, we do indeed retain the null hypothesis.  We could not find enough evidence that the signup rates for Mailer 1 and Mailer 2 were different - and thus conclude that there was no significant difference.
null hypothesis

While we saw that the higher cost Mailer 2 had a higher signup rate (37.8%) than the lower cost Mailer 1 (32.8%) it appears that this difference is not significant, at least at our Acceptance Criteria of 0.05.

Without running this Hypothesis Test, the client may have concluded that they should always look to go with higher cost mailers - and from what we've seen in this test, that may not be a great decision.  It would result in them spending more, but not *necessarily* gaining any extra revenue as a result

Our results here also do not say that there *definitely isn't a difference between the two mailers* - we are only advising that we should not make any rigid conclusions *at this point*.  

Running more A/B Tests like this, gathering more data, and then re-running this test may provide us, and the client more insight!
